## 1. 載入資料並確認thread_id 符合定義

> 刪除 src_cre_date 重複的組別後，( item_id,byr_id,slr_id ) 無論是否唯一，都只會對應到唯一一個thread<br>
> 上述刪除採用cascate (只要找到一筆，則整組( item_id,byr_id,slr_id )一同刪除 )<br>
> =>為甚麼不是distinct ? 因為用 src_cre_date 刪除更為嚴格也符合常理
> 僅有少數組資料記錄錯誤，因此將其刪除

In [1]:
import polars as pl

pl_df = pl.read_csv(r'C:\Users\ChenChun\Downloads\bargaining_data\anon_bo_threads.csv'); 

original_count = pl_df.shape[0]
print(f'初始資料筆數: {pl_df.shape[0]}')

初始資料筆數: 47377200


In [2]:
# 找出 src_cre_date 完全重複的組合（賣家+買家+商品+時間）
dup_keys = (
    pl_df
    .group_by(['anon_slr_id', 'anon_byr_id', 'anon_item_id', 'src_cre_date'])
    .agg(pl.count().alias("count"))
    .filter(pl.col("count") > 1)
    .select(['anon_slr_id', 'anon_byr_id', 'anon_item_id', 'src_cre_date'])
)

# Step 3：找出所有屬於這些重複行的原始資料
duplicated_rows = pl_df.join(
    dup_keys,
    on=['anon_slr_id', 'anon_byr_id', 'anon_item_id', 'src_cre_date'],
    how='inner'
)
# 印出範例資料（前 5 筆）
print("========== 重複 src_cre_date 的原始資料範例（前 5 筆）：")
print(duplicated_rows.select([
    'anon_slr_id', 'anon_byr_id', 'anon_item_id','anon_thread_id' , 'src_cre_date','offr_type_id','status_id','response_time'
]).head(5))

# Step 4：從這些 row 找出該整組的 item group（不含時間）
groups_to_remove = duplicated_rows.select([
    'anon_slr_id', 'anon_byr_id', 'anon_item_id'
]).unique()

# Step 5：從原始 df 中刪掉整組 group
df_cleaned = pl_df.join(
    groups_to_remove,
    on=['anon_slr_id', 'anon_byr_id', 'anon_item_id'],
    how='anti'
)

# Step 6：統計刪除筆數
deleted_rows_count = original_count - df_cleaned.shape[0]
print("刪除整組（因 src_cre_date 重複）之筆數:", deleted_rows_count)


C:\Users\ChenChun\AppData\Local\Temp\ipykernel_8336\3172605028.py:5: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  .agg(pl.count().alias("count"))


========== 重複 src_cre_date 的原始資料範例（前 5 筆）：
shape: (5, 8)
┌────────────┬────────────┬────────────┬───────────┬───────────┬───────────┬───────────┬───────────┐
│ anon_slr_i ┆ anon_byr_i ┆ anon_item_ ┆ anon_thre ┆ src_cre_d ┆ offr_type ┆ status_id ┆ response_ │
│ d          ┆ d          ┆ id         ┆ ad_id     ┆ ate       ┆ _id       ┆ ---       ┆ time      │
│ ---        ┆ ---        ┆ ---        ┆ ---       ┆ ---       ┆ ---       ┆ i64       ┆ ---       │
│ i64        ┆ i64        ┆ i64        ┆ i64       ┆ str       ┆ i64       ┆           ┆ str       │
╞════════════╪════════════╪════════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 1395337    ┆ 6282576    ┆ 76566287   ┆ 4376595   ┆ 18jul2013 ┆ 0         ┆ 2         ┆ 18jul2013 │
│            ┆            ┆            ┆           ┆ 08:52:26  ┆           ┆           ┆ 09:34:53  │
│ 1395337    ┆ 6282576    ┆ 76566287   ┆ 4376595   ┆ 18jul2013 ┆ 0         ┆ 2         ┆ 18jul2013 │
│            ┆            ┆       

In [3]:
# 先記錄原始筆數
original_count = df_cleaned.shape[0]

# ---------------------
# 第一段：精準條件刪除 (包含 item_id, thread_id, byr_id, slr_id)
# ---------------------
to_delete = [
    (58639544, 8655052, 5712605, 3943399),
    (58639544, 1632051, 5712605, 3943399),
    (2773744, 7212858, 1689684, 5718581),
    (2773744, 3721458, 1689684, 5718581),
]

# 轉成 Polars 的條件過濾邏輯
for item_id, thread_id, byr_id, slr_id in to_delete:
    df_cleaned = df_cleaned.filter(~(
        (pl.col("anon_item_id") == item_id) &
        (pl.col("anon_thread_id") == thread_id) &
        (pl.col("anon_byr_id") == byr_id) &
        (pl.col("anon_slr_id") == slr_id)
    ))

# ---------------------
# 第二段：以 (slr_id, byr_id, item_id) 為 key 刪除整組
# ---------------------
search_conditions = [
    (805737, 3547513, 9145419),
    (4667211, 8350361, 40000809),
    (7773303, 3771453, 47886959),
    (7927663, 3576898, 67593548),
    (9211073, 9334414, 88204993),
    (9537923, 7605857, 40868572),
    (10176360, 8235394, 59174743),
]

# 建立要刪除的組合 DataFrame
conditions_df = pl.DataFrame(search_conditions, schema=["anon_slr_id", "anon_byr_id", "anon_item_id"])

# 使用 anti join 刪除這些 group
df_cleaned = df_cleaned.join(
    conditions_df,
    on=["anon_slr_id", "anon_byr_id", "anon_item_id"],
    how="anti"
)

# ---------------------
# 印出結果
# ---------------------
deleted_count = original_count - df_cleaned.shape[0]
print("共刪除", deleted_count, "筆例外資料（這些組別沒有唯一的 thread_id）")
print("====================================")
print("目前的資料筆數: ", df_cleaned.shape[0])


C:\Users\yuwenhsuuu\anaconda3\lib\functools.py:889: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return dispatch(args[0].__class__)(*args, **kw)


共刪除 15 筆例外資料（這些組別沒有唯一的 thread_id）
目前的資料筆數:  47361446


> 檢查上述程式碼是否執行正確
> 檢查 groupby(slr, byr, item) 後的資料是否僅有唯一 thread_id

In [4]:
# 針對每組 (slr_id, byr_id, item_id) 統計 thread_id 的不重複數量
grouped = (
    df_cleaned
    .group_by(['anon_slr_id', 'anon_byr_id', 'anon_item_id'])
    .agg(pl.col('anon_thread_id').n_unique().alias('thread_id_count'))
)

# 篩選出 thread_id 不唯一的組
non_unique_threads = grouped.filter(pl.col('thread_id_count') > 1)

# 印出結果
if non_unique_threads.is_empty():
    print("✅ 所有組合的 thread_id 都是唯一的。")
else:
    print("⚠️ 以下組合的 thread_id 不唯一：")
    print(non_unique_threads)



✅ 所有組合的 thread_id 都是唯一的。


### 2.填值 
買賣雙方針對商品產生的討論串中，有些欄位的值只會出現在第一筆<br>
例如: groupby(slr, byr, item, thread)， slr_hist/byr_hist/byr_us 正確的值僅存於組內排序後的第一筆資料(用src_cre_date排序，而非offr_type_id==0)，所以要先做填值，才能做資料清理([ppt第三十頁](https://docs.google.com/presentation/d/1b9MVElz9fjTnk3c-D-jzDurlGD0HzqPzE8fCyvo7NDI/edit#slide=id.g33877aea87e_0_2))



> **注意:正確排序需要先將字串轉換為日期時間格式**

In [5]:
df_cleaned = df_cleaned.with_columns([
    pl.col("src_cre_date").str.strptime(pl.Datetime, format="%d%b%Y %H:%M:%S").alias("src_cre_date"),
    pl.col("response_time").filter(pl.col("response_time").is_not_null())
      .str.strptime(pl.Datetime, format="%d%b%Y %H:%M:%S").alias("response_time")
])

> 先群組並組內排序 : <br>
> 1. 檢查組內第一筆資料使否offer_type_id ==0
> 2. 檢查補值欄位 'slr_hist', 'byr_hist', 'byr_us' 是否為空值
> 3. 發現 slr_hist 具空值欄位 僅組內一筆資料時，將 null值補 0 (因為slr_hist最小為1)
> 4. 填值: 用組內排序後的第一筆資料(用src_cre_date排序，而非offr_type_id==0)，做填值

In [6]:
df_cleaned.shape[0]

47361446

In [7]:
# 欲補值欄位
cols_to_fill = ['slr_hist', 'byr_hist', 'byr_us']  # 定義需要填補的欄位
group_keys = ['anon_slr_id', 'anon_byr_id', 'anon_item_id', 'anon_thread_id']  # 定義分組的鍵值


# 先排序，然後添加行號
df_ranked = (
    df_cleaned  # 包含日期時間格式的資料框
    .sort(group_keys + ["src_cre_date"])  # 依照群組鍵和時間排序
    .with_columns([
        pl.col("offr_type_id"),
        *[pl.col(col) for col in cols_to_fill],
        # 使用over()函數在每個群組中創建排名
        pl.col("src_cre_date").rank("dense").over(group_keys).alias("row_num")
    ])
)

#  ======== 1. 檢查組內第一筆資料使否offer_type_id ==0 
df_first_rows = df_ranked.filter(pl.col("row_num") == 1)  # 篩選出每個群組的第一筆資料
df_offr_not_zero = df_first_rows.filter(pl.col("offr_type_id") != 0)  # 找出不符合預期的第一筆（非買家出價）
count_offr_not_zero = df_offr_not_zero.shape[0]  # 計算這類例外資料的數量
print(f"\noffr_type_id != 0 的第一筆資料有 {count_offr_not_zero} 筆")

#  ======== 2. 檢查補值欄位 'slr_hist', 'byr_hist', 'byr_us' 是否為空值
null_check = df_first_rows.select([
    pl.lit(df_first_rows.shape[0]).alias("總資料筆數"),# 計算總共有多少個第一筆資料
    
    # 檢查每個欄位是否有 null
    pl.col("slr_hist").is_null().sum().alias("slr_hist_null_count"),
    pl.col("byr_hist").is_null().sum().alias("byr_hist_null_count"),
    pl.col("byr_us").is_null().sum().alias("byr_us_null_count"),
    
])

print("\n第一筆資料的空值檢查結果:")
print(null_check)

# # 印出第一筆資料中slr_hist為空的前幾筆記錄
# null_slr_hist_first = df_first_rows.filter(pl.col("slr_hist").is_null())
# print("\nslr_hist為空的第一筆資料 (前5筆):")
# print(null_slr_hist_first.select(group_keys + ["src_cre_date", "slr_hist", "offr_type_id"]).head(5))


# 將 slr_hist為null的資料 填補 0
df_cleaned_filled = df_ranked.with_columns(
    pl.when(pl.col("slr_hist").is_null() & (pl.col("row_num") == 1)) # 檢查每一筆資料中的slr_hist欄位是否為空值，是否為其所屬群組的第一筆資料（row_num == 1）
      .then(pl.lit(0))
      .otherwise(pl.col("slr_hist"))
      .alias("slr_hist")
)



offr_type_id != 0 的第一筆資料有 0 筆

第一筆資料的空值檢查結果:
shape: (1, 4)
┌────────────┬─────────────────────┬─────────────────────┬───────────────────┐
│ 總資料筆數 ┆ slr_hist_null_count ┆ byr_hist_null_count ┆ byr_us_null_count │
│ ---        ┆ ---                 ┆ ---                 ┆ ---               │
│ i32        ┆ u32                 ┆ u32                 ┆ u32               │
╞════════════╪═════════════════════╪═════════════════════╪═══════════════════╡
│ 28197270   ┆ 840080              ┆ 0                   ┆ 0                 │
└────────────┴─────────────────────┴─────────────────────┴───────────────────┘


> 檢查第一筆資料 slr_hist、byr_hist、byr_us 三欄位的分布


In [8]:

df_first_rows = df_cleaned_filled.filter(pl.col("row_num") == 1)  # 篩選出每個群組的第一筆資料
null_check_after = df_first_rows.select([
    pl.lit(df_first_rows.shape[0]).alias("總資料筆數"),
    pl.col("slr_hist").is_null().sum().alias("slr_hist_null_count"),
    pl.col("byr_hist").is_null().sum().alias("byr_hist_null_count"),
    pl.col("byr_us").is_null().sum().alias("byr_us_null_count")
])

print("第一筆三欄位空值情況：")
print(null_check_after)

# 計算三個欄位的基本統計量
stats_first_rows = df_first_rows.select([
    pl.lit(df_first_rows.shape[0]).alias("總資料筆數"),
    
    # slr_hist 統計
    pl.col("slr_hist").mean().alias("slr_hist_平均值"),
    # pl.col("slr_hist").median().alias("slr_hist_中位數"),
    # pl.col("slr_hist").min().alias("slr_hist_最小值"),
    # pl.col("slr_hist").max().alias("slr_hist_最大值"),
    # pl.col("slr_hist").std().alias("slr_hist_標準差"),
    
    # byr_hist 統計
    pl.col("byr_hist").mean().alias("byr_hist_平均值"),
    # pl.col("byr_hist").median().alias("byr_hist_中位數"),
    # pl.col("byr_hist").min().alias("byr_hist_最小值"),
    # pl.col("byr_hist").max().alias("byr_hist_最大值"),
    # pl.col("byr_hist").std().alias("byr_hist_標準差"),
    
    # byr_us 統計
    pl.col("byr_us").mean().alias("byr_us_平均值"),
    # pl.col("byr_us").median().alias("byr_us_中位數"),
    # pl.col("byr_us").min().alias("byr_us_最小值"),
    # pl.col("byr_us").max().alias("byr_us_最大值"),
    # pl.col("byr_us").std().alias("byr_us_標準差")
])

print("第一筆資料三欄位的基本統計：")
print(stats_first_rows)


第一筆三欄位空值情況：
shape: (1, 4)
┌────────────┬─────────────────────┬─────────────────────┬───────────────────┐
│ 總資料筆數 ┆ slr_hist_null_count ┆ byr_hist_null_count ┆ byr_us_null_count │
│ ---        ┆ ---                 ┆ ---                 ┆ ---               │
│ i32        ┆ u32                 ┆ u32                 ┆ u32               │
╞════════════╪═════════════════════╪═════════════════════╪═══════════════════╡
│ 28197270   ┆ 0                   ┆ 0                   ┆ 0                 │
└────────────┴─────────────────────┴─────────────────────┴───────────────────┘
第一筆資料三欄位的基本統計：
shape: (1, 4)
┌────────────┬─────────────────┬─────────────────┬───────────────┐
│ 總資料筆數 ┆ slr_hist_平均值 ┆ byr_hist_平均值 ┆ byr_us_平均值 │
│ ---        ┆ ---             ┆ ---             ┆ ---           │
│ i32        ┆ f64             ┆ f64             ┆ f64           │
╞════════════╪═════════════════╪═════════════════╪═══════════════╡
│ 28197270   ┆ 3746.670963     ┆ 132.362123      ┆ 0.873565      │
└────────

> 用組內排序後第一筆資料對組內後續資料作填值

In [9]:
# 用組內排序後第一筆資料對組內後續資料作填值

# 先獲取每個群組的第一筆資料
first_values_per_group = df_cleaned_filled.filter(pl.col("row_num") == 1).select([
    *group_keys,  # 群組鍵
    pl.col("slr_hist").alias("first_slr_hist"),
    pl.col("byr_hist").alias("first_byr_hist"),
    pl.col("byr_us").alias("first_byr_us")
])

# 將第一筆資料與原始數據連接
df_with_first = df_cleaned_filled.join(
    first_values_per_group,
    on=group_keys,
    how="left"
)

# 使用第一筆資料填充空值 (為所有組內資料)
df_filled = df_with_first.with_columns([
    # 若slr_hist為空，則使用該組第一筆的值
    pl.when(pl.col("slr_hist").is_null())
      .then(pl.col("first_slr_hist"))
      .otherwise(pl.col("slr_hist"))
      .alias("slr_hist"),
    
    # 若byr_hist為空，則使用該組第一筆的值
    pl.when(pl.col("byr_hist").is_null())
      .then(pl.col("first_byr_hist"))
      .otherwise(pl.col("byr_hist"))
      .alias("byr_hist"),
    
    # byr_us欄位直接用第一筆資料覆蓋所有資料
    pl.col("first_byr_us").alias("byr_us")
])

# 移除臨時列
df_filled_final = df_filled.drop(["first_slr_hist", "first_byr_hist", "first_byr_us"])


# 檢查填充後是否還有空值
null_check_after = df_filled_final.select([
    pl.lit(df_filled_final.shape[0]).alias("總資料筆數"),
    # pl.col("first_slr_hist").is_null().sum().alias("new_slr_hist_null_count"),
    pl.col("slr_hist").is_null().sum().alias("slr_hist_null_count"),
    # pl.col("first_byr_hist").is_null().sum().alias("new_byr_hist_null_count"),
    pl.col("byr_hist").is_null().sum().alias("byr_hist_null_count"),
    # pl.col("first_byr_us").is_null().sum().alias("new_byr_us_null_count"),
    pl.col("byr_us").is_null().sum().alias("byr_us_null_count")
])

print("\n填充後的空值檢查結果:")
print(null_check_after)



填充後的空值檢查結果:
shape: (1, 4)
┌────────────┬─────────────────────┬─────────────────────┬───────────────────┐
│ 總資料筆數 ┆ slr_hist_null_count ┆ byr_hist_null_count ┆ byr_us_null_count │
│ ---        ┆ ---                 ┆ ---                 ┆ ---               │
│ i32        ┆ u32                 ┆ u32                 ┆ u32               │
╞════════════╪═════════════════════╪═════════════════════╪═══════════════════╡
│ 47361446   ┆ 0                   ┆ 0                   ┆ 0                 │
└────────────┴─────────────────────┴─────────────────────┴───────────────────┘


## 篩出 同一買賣家 在相近時間(48hr內)談判不同商品

> 確認 src_cre_date 和 response_time 為 datetime

In [10]:
src_cre_date_type = df_filled_final.select(pl.col("src_cre_date")).dtypes[0]
response_time_type = df_filled_final.select(pl.col("response_time")).dtypes[0]

print(f"src_cre_date的數據類型：{src_cre_date_type}")
print(f"response_time的數據類型：{response_time_type}")

src_cre_date的數據類型：Datetime(time_unit='us', time_zone=None)
response_time的數據類型：Datetime(time_unit='us', time_zone=None)


> 排序 -> 計算相鄰兩筆資料的時間差 -> 標註相鄰兩筆48hr內資料overlap -> 將被標註過的組移出 df -> 存進"整理.csv"、"被刪除.csv"

In [11]:
def debug_overlap_window(df: pl.DataFrame, buffer: int = 1) -> pl.DataFrame:
    """
    顯示所有 final_overlap = 1 的紀錄，並包含前後 buffer 筆資料，用來人工檢查 overlap 標註是否正確。
    
    參數：
        df     : 已經有 final_overlap 欄位的資料表
        buffer : 要顯示多少前後相鄰紀錄（預設為 1）

    回傳：
        pl.DataFrame：包含 overlap 區段前後紀錄的子集（已去除重複）
    """
    # 加入群內排序編號
    df_debug = df.with_columns([
        pl.arange(0, pl.count()).over(["anon_slr_id", "anon_byr_id"]).alias("row_in_group")
    ])

    # 找出有 final_overlap 的行
    rows_with_overlap = df_debug.filter(pl.col("final_overlap") == 1).select([
        "anon_slr_id", "anon_byr_id", "row_in_group"
    ])

    # 加上前後 buffer 區間
    buffered_rows = rows_with_overlap.with_columns([
        (pl.col("row_in_group") - buffer).alias("start_row"),
        (pl.col("row_in_group") + buffer).alias("end_row")
    ])

    # 抓出前後 buffer 內的紀錄
    df_to_check = (
        df_debug.join(
            buffered_rows,
            on=["anon_slr_id", "anon_byr_id"],
            how="inner"
        )
        .filter(
            (pl.col("row_in_group") >= pl.col("start_row")) &
            (pl.col("row_in_group") <= pl.col("end_row"))
        )
        .select([
            "anon_slr_id", "anon_byr_id", "anon_item_id",
            "row_in_group", "src_cre_date", "prev_time",
            "time_diff_seconds", "item_changed",
            "overlap", "next_overlap", "final_overlap"
        ])
        .unique()  # ✅ 去除重複筆數
        .sort(["anon_slr_id", "anon_byr_id", "row_in_group"])
    )

    return df_to_check


In [12]:
from polars.datatypes import Struct, List

# Step 1: 排序並計算相鄰紀錄差異
"""
第一步驟 : 
    每個 (slr_id, byr_id) 的紀錄會根據 src_cre_date 升冪排列（早→晚）
第二步驟 :
    針對每組 slr/byr 抓取：
    上一筆的 src_cre_date → prev_time
    上一筆的 item_id → prev_item
第三步驟 :
    標註兩筆之間是否為短時間內換商品 : 
    這裡是抓出與前一筆相差幾秒（若同一 slr/byr）
    然後標記 item_id 是否不同 → item_changed
第四步驟 :
    定義 overlap 條件 : 
    48 * 3600 是 48 小時
    在 48 小時內出現 不同商品 的紀錄，就標記為 
    .shift(1) 的設計下  overlap 會「滯後標記」，只在pair 的後一筆被標記為 overlap
"""
df_sorted = (
    # 第一步驟
    df_filled_final.sort(["anon_slr_id", "anon_byr_id", "src_cre_date"]) 
    # 第二步驟
    .with_columns([
        pl.col("src_cre_date").shift(1).over(["anon_slr_id", "anon_byr_id"]).alias("prev_time"),
        pl.col("anon_item_id").shift(1).over(["anon_slr_id", "anon_byr_id"]).alias("prev_item"),
    ])
    # 第三步驟
    .with_columns([
        (pl.col("src_cre_date") - pl.col("prev_time")).dt.total_seconds().alias("time_diff_seconds"),
        (pl.col("anon_item_id") != pl.col("prev_item")).alias("item_changed")
    ])
    # 第四步驟
    .with_columns([
        pl.when(
            (pl.col("time_diff_seconds") <= 48 * 3600) & pl.col("item_changed")
        ).then(1).otherwise(0).alias("overlap")
    ])
)

# 驗證: 只有每組的第一筆，prev_time 才應該是 null ===============================
# df_checked = df_sorted.with_columns([
#     pl.arange(0, pl.count()).over(["anon_slr_id", "anon_byr_id"]).alias("row_in_group")
# ])
# invalid_nulls = df_checked.filter(
#     (pl.col("row_in_group") > 0) & (pl.col("prev_time").is_null())
# )
# print("非第一筆但 prev_time 為 null 的異常筆數：", invalid_nulls.height)
# if invalid_nulls.height > 0:
#     print(invalid_nulls.select([
#         "anon_slr_id", "anon_byr_id", "src_cre_date", "row_in_group", "prev_time"
#     ]).limit(5))
# else:
#     print("✅驗證成功：只有每組的第一筆 prev_time 才為 null")
# ==============================================================================



# Step 2: 標註相鄰也可能 overlap 的紀錄
# 補回漏標的 overlap pair 前一筆 (被標記 overlap 的前一筆也應該是 overlap)
df_overlap = df_sorted.with_columns([
    pl.col("overlap").shift(-1).over(["anon_slr_id", "anon_byr_id"]).fill_null(0).alias("next_overlap")
]).with_columns([
    (pl.col("overlap") | pl.col("next_overlap")).alias("final_overlap")
])

# 驗證: 抓出「包含 overlap 附近的資料段落」 =====================
debug_result = debug_overlap_window(df_overlap)# 顯示 overlap 周圍前後 1 筆（預設值）
print(debug_result.limit(20))# 印出前 20 筆檢查
# =============================================================

# Step 3: 找出有 overlap 的小組 (slr/byr/item)
# 最終結果是：所有出現 overlap 的「(賣家、買家、商品) 組合」
overlap_items = (
    df_overlap.filter(pl.col("final_overlap") == 1)
    .select(["anon_slr_id", "anon_byr_id", "anon_item_id"])
    .unique()
)
print(f"被移除的小組數量（item 數）：{overlap_items.height}")
print(f"曾在短期內參與過不同 item 交易的 slr/byr 對數量：{overlap_items.select(['anon_slr_id', 'anon_byr_id']).unique().height}")

# Step 4: 從原始資料中移除這些小組
original_count = df_overlap.height  # 原始筆數
df_final = df_overlap.join(overlap_items, on=["anon_slr_id", "anon_byr_id", "anon_item_id"], how="anti")
removed_count = original_count - df_final.height  # 計算移除筆數


# 匯出處理後的資料
df_final.write_csv("data/filtered_records.csv")

# 印出處理資訊
print(f"檔案已儲存於 filtered_records.csv，共保留 {df_final.height} 筆")
print(f"已移除違規資料筆數：{removed_count} 筆")



C:\Users\ChenChun\AppData\Local\Temp\ipykernel_8336\577357169.py:14: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.arange(0, pl.count()).over(["anon_slr_id", "anon_byr_id"]).alias("row_in_group")


shape: (20, 11)
┌────────────┬───────────┬───────────┬───────────┬───┬───────────┬─────────┬───────────┬───────────┐
│ anon_slr_i ┆ anon_byr_ ┆ anon_item ┆ row_in_gr ┆ … ┆ item_chan ┆ overlap ┆ next_over ┆ final_ove │
│ d          ┆ id        ┆ _id       ┆ oup       ┆   ┆ ged       ┆ ---     ┆ lap       ┆ rlap      │
│ ---        ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ i32     ┆ ---       ┆ ---       │
│ i64        ┆ i64       ┆ i64       ┆ i64       ┆   ┆ bool      ┆         ┆ i32       ┆ i32       │
╞════════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═════════╪═══════════╪═══════════╡
│ 89         ┆ 589814    ┆ 54955850  ┆ 0         ┆ … ┆ null      ┆ 0       ┆ 1         ┆ 1         │
│ 89         ┆ 589814    ┆ 24454497  ┆ 1         ┆ … ┆ true      ┆ 1       ┆ 0         ┆ 1         │
│ 89         ┆ 589814    ┆ 24454497  ┆ 2         ┆ … ┆ false     ┆ 0       ┆ 0         ┆ 0         │
│ 140        ┆ 4470202   ┆ 64190767  ┆ 0         ┆ … ┆ null      ┆ 0       

> 反驗證

In [13]:
# 找出時間差異接近48小時邊界的案例
boundary_cases = df_sorted.filter(
    (pl.col("time_diff_seconds").is_not_null()) & 
    (pl.col("time_diff_seconds") > 48 * 3600 - 3600) & 
    (pl.col("time_diff_seconds") < 48 * 3600 + 3600) &
    (pl.col("item_changed"))
).select([
    "anon_slr_id", "anon_byr_id", "anon_item_id", 
    "src_cre_date", "prev_time", "time_diff_seconds",
    "overlap"
]).sort("time_diff_seconds")

print("==== 時間差異接近48小時邊界的案例 ====")
print(boundary_cases.limit(10))

# 檢視時間差異的統計信息
time_diff_summary = df_sorted.filter(
    pl.col("time_diff_seconds").is_not_null() & pl.col("item_changed")
).select([
    pl.min("time_diff_seconds").alias("最小時間差(秒)"),
    pl.max("time_diff_seconds").alias("最大時間差(秒)"),
    pl.mean("time_diff_seconds").alias("平均時間差(秒)")
])

print("==== 時間差異統計 ====")
print(time_diff_summary)

==== 時間差異接近48小時邊界的案例 ====
shape: (10, 7)
┌─────────────┬─────────────┬──────────────┬───────────────┬──────────────┬──────────────┬─────────┐
│ anon_slr_id ┆ anon_byr_id ┆ anon_item_id ┆ src_cre_date  ┆ prev_time    ┆ time_diff_se ┆ overlap │
│ ---         ┆ ---         ┆ ---          ┆ ---           ┆ ---          ┆ conds        ┆ ---     │
│ i64         ┆ i64         ┆ i64          ┆ datetime[μs]  ┆ datetime[μs] ┆ ---          ┆ i32     │
│             ┆             ┆              ┆               ┆              ┆ i64          ┆         │
╞═════════════╪═════════════╪══════════════╪═══════════════╪══════════════╪══════════════╪═════════╡
│ 3054028     ┆ 9693622     ┆ 63475063     ┆ 2013-05-03    ┆ 2013-05-01   ┆ 169201       ┆ 1       │
│             ┆             ┆              ┆ 11:41:01      ┆ 12:41:00     ┆              ┆         │
│ 3622812     ┆ 6354757     ┆ 81113427     ┆ 2013-02-11    ┆ 2013-02-09   ┆ 169201       ┆ 1       │
│             ┆             ┆              ┆ 07:35

## df_cleaned 處理空值
any_mssg、'fdbk_score_src', 'fdbk_pstv_src' 若為空值，補 0


In [14]:
import polars as pl
df_final = pl.read_csv('data/filtered_records.csv')
print(df_final.shape)

null_counts = df_final.null_count()
cols_with_nulls = [col for col, count in zip(null_counts.columns, null_counts.row(0)) if count != 0]
print(f"欄位有缺失值：{cols_with_nulls}")

print(df_final.columns)
df_final = df_final.drop(['prev_time', 'prev_item', 'time_diff_seconds', 'item_changed', 'overlap', 'next_overlap', 'final_overlap'])
print(df_final.columns)

(40824367, 24)
欄位有缺失值：['fdbk_score_src', 'fdbk_pstv_src', 'any_mssg', 'prev_time', 'prev_item', 'time_diff_seconds', 'item_changed']
['anon_item_id', 'anon_thread_id', 'anon_byr_id', 'anon_slr_id', 'src_cre_dt', 'fdbk_score_src', 'fdbk_pstv_src', 'offr_type_id', 'status_id', 'offr_price', 'src_cre_date', 'response_time', 'slr_hist', 'byr_hist', 'any_mssg', 'byr_us', 'row_num', 'prev_time', 'prev_item', 'time_diff_seconds', 'item_changed', 'overlap', 'next_overlap', 'final_overlap']
['anon_item_id', 'anon_thread_id', 'anon_byr_id', 'anon_slr_id', 'src_cre_dt', 'fdbk_score_src', 'fdbk_pstv_src', 'offr_type_id', 'status_id', 'offr_price', 'src_cre_date', 'response_time', 'slr_hist', 'byr_hist', 'any_mssg', 'byr_us', 'row_num']


In [15]:

# 將指定欄位的空值補為 0
df_final = df_final.with_columns([
    pl.col('any_mssg').fill_null(0),
    pl.col('fdbk_score_src').fill_null(0),
    pl.col('fdbk_pstv_src').fill_null(0)
])

# 查看更新後的結果
print(df_final.select(['any_mssg', 'fdbk_score_src', 'fdbk_pstv_src']).describe())


shape: (9, 4)
┌────────────┬─────────────┬────────────────┬───────────────┐
│ statistic  ┆ any_mssg    ┆ fdbk_score_src ┆ fdbk_pstv_src │
│ ---        ┆ ---         ┆ ---            ┆ ---           │
│ str        ┆ f64         ┆ f64            ┆ f64           │
╞════════════╪═════════════╪════════════════╪═══════════════╡
│ count      ┆ 4.0824367e7 ┆ 4.0824367e7    ┆ 4.0824367e7   │
│ null_count ┆ 0.0         ┆ 0.0            ┆ 0.0           │
│ mean       ┆ 0.135966    ┆ 6221.791541    ┆ 99.120297     │
│ std        ┆ 0.342753    ┆ 22789.566126   ┆ 7.368475      │
│ min        ┆ 0.0         ┆ -2.0           ┆ 0.0           │
│ 25%        ┆ 0.0         ┆ 302.0          ┆ 99.6          │
│ 50%        ┆ 0.0         ┆ 1223.0         ┆ 99.85         │
│ 75%        ┆ 0.0         ┆ 4451.0         ┆ 100.0         │
│ max        ┆ 1.0         ┆ 2.299731e6     ┆ 100.0         │
└────────────┴─────────────┴────────────────┴───────────────┘


## 刪除 status_id ==6 or status_id ==9 的資料
- 不能直接刪掉!!! 要判斷 offer_type_id == 0 時，才可刪除。

In [16]:
# df_final 已是 Polars DataFrame
pl_df = df_final.with_columns(
    pl.concat_str(["anon_item_id", "anon_thread_id", "anon_byr_id", "anon_slr_id"], separator="_").alias("group_key")
)


### ===== 操作一：offr_type_id == 0 且 status_id in [6, 9] =====
original_count = pl_df.height

trigger_rows = pl_df.filter(
    (pl.col("offr_type_id") == 0) & (pl.col("status_id").is_in([6, 9]))
)
trigger_count = trigger_rows.height

keys_to_delete = trigger_rows.select("group_key").unique().to_series().to_list()

pl_df = pl_df.filter(~pl.col("group_key").is_in(keys_to_delete))
new_count = pl_df.height
deleted_count = original_count - new_count


print(f"===== 操作一：offr_type_id == 0 且 status_id in [6, 9] (自動接受/刪除)=====")
print(f"觸發筆數: {trigger_count}，整組刪除: {deleted_count}，剩餘: {new_count}\n")

### ===== 操作二：status_id in [1, 2, 8] 且 src_cre_date == response_time =====
original_count = pl_df.height

trigger_rows = pl_df.filter(
    (pl.col("status_id").is_in([1, 2, 8])) & (pl.col("src_cre_date") == pl.col("response_time"))
)
trigger_count = trigger_rows.height

keys_to_delete = trigger_rows.select("group_key").unique().to_series().to_list()

pl_df = pl_df.filter(~pl.col("group_key").is_in(keys_to_delete))
new_count = pl_df.height
deleted_count = original_count - new_count

print(f"===== 操作二：status_id in [1, 2, 8] 且 src_cre_date == response_time =====")
print(f"觸發筆數: {trigger_count}，整組刪除: {deleted_count}，剩餘: {new_count}\n")

### ===== 操作三：src_cre_date > response_time =====
original_count = pl_df.height

trigger_rows = pl_df.filter(
    pl.col("src_cre_date") > pl.col("response_time")
)
trigger_count = trigger_rows.height

keys_to_delete = trigger_rows.select("group_key").unique().to_series().to_list()

pl_df = pl_df.filter(~pl.col("group_key").is_in(keys_to_delete))
new_count = pl_df.height
deleted_count = original_count - new_count

print(f"===== 操作三：src_cre_date > response_time =====")
print(f"觸發筆數: {trigger_count}，整組刪除: {deleted_count}，剩餘: {new_count}")

### 最後移除輔助欄位
pl_df = pl_df.drop("group_key")


===== 操作一：offr_type_id == 0 且 status_id in [6, 9] (自動接受/刪除)=====
觸發筆數: 8753678，整組刪除: 10006674，剩餘: 30817693

===== 操作二：status_id in [1, 2, 8] 且 src_cre_date == response_time =====
觸發筆數: 264，整組刪除: 300，剩餘: 30817393

===== 操作三：src_cre_date > response_time =====
觸發筆數: 727，整組刪除: 911，剩餘: 30816482


# 篩選 round 1 和 round 2 的資料

> 0320 改成: 第一輪就結束的資料才屬於round1 ； 只有兩輪的資料才屬於round2

1. 資料以此四個欄位分群: ["anon_item_id", "anon_slr_id", "anon_thread_id", "anon_byr_id"]  並依 src_cre_date 排序 
2. 篩選組內僅有 offr_type_id == 0 的資料，這些資料存入 r1_df 
3. 接著在剩餘的資料中，篩選每一群中 最後一個offr_type_id == 0 (前面已使用src_cre_date排序)的資料的下一筆(offr_type_id != 0) ，若是此資料只有一筆offr_type_id != 0接續在組內最後一筆offr_type_id == 0 之後，則將最後一個 0 的那筆以及下一筆非0 存入 r2_df<br>
*註: 若 組內最後一筆offr_type_id == 0 之後有大於一筆offr_type_id != 0 即**不能 存入 r2_df**
*註: 最後一筆 0 跟第一筆非 0 之前的所有資料 → 它們才是我們要存進 r2_df 的！
    範例資料: 
    ```
    group	src_cre_date	offr_type_id	row_number
    A	    2024-01-01	    0	    	    0
    A	    2024-01-02	    0	    	    1
    A	    2024-01-03	    1	   	      2
    # 這三筆都應該存進 r2_df ，但先前我們山除了自動接受跟拒絕，
    # 因此這種屬於例外，應查看有幾筆 (也就是r2_df 中有幾組group 是超過2筆資料的)
    ```

4. 顯示r1_df, r2_df 各有幾筆資料，及沒有被分入r1_df, r2_df 的pl_df 剩下的有幾筆資料

In [17]:


# 分組依據的欄位
group_cols = ["anon_item_id", "anon_slr_id", "anon_thread_id", "anon_byr_id"]

# Step 1: 先根據分組欄位與日期欄位進行排序(組內排序)
pl_df = pl_df.sort(group_cols + ["src_cre_date"])

# Step 2: 找出每組中只包含 offr_type_id == 0 的群組
# 方法：計算每組中 offr_type_id 的最大值
type_stats = (
    pl_df
    .group_by(group_cols)
    .agg([
        pl.col("offr_type_id").max().alias("max_type_id"),
        pl.col("offr_type_id").min().alias("min_type_id"),
        pl.len().alias("count_rows")
    ])
)

# 找出整組都是 0 的群組條件：max == 0 且 min == 0 ， 得到一張「只有 group key 的 DataFrame」
only_zero_groups = type_stats.filter(
    (pl.col("max_type_id") == 0) & (pl.col("min_type_id") == 0)
).select(group_cols)

# 用 join 方式篩選這些群組的資料 → r1_df
r1_df = pl_df.join(only_zero_groups, on=group_cols, how="inner")

# Step 3: 剩下的資料 = 非純 0 的群組
remaining_df = pl_df.join(only_zero_groups, on=group_cols, how="anti") # anti 會跟 inner互補

# Step 3-1: 為了進一步處理，我們為每筆資料加上該群內的 row number
# 方便後面定位特定位置
remaining_with_rownum = (
    remaining_df
    .with_columns([
        pl.col("offr_type_id").alias("original_type"),  # 保留原始 type 資訊
        pl.arange(0, pl.len()).over(group_cols).alias("row_number"),
    ])
)

# Step 3-2: 抓出每組中最後一筆 offr_type_id == 0 的 row_number
last_zero_index = (
    remaining_with_rownum
    .filter(pl.col("original_type") == 0)
    .group_by(group_cols)
    .agg(pl.col("row_number").max().alias("last_zero_row"))
)

# ====================== 補充檢查 =========================
# 檢查 group 中「第一筆為 type != 0」後，是否仍出現過 type == 0
# 印出違反此規則的資料筆數 : (若沒有違反則不會print]違反則不會print])

# Step A: 先找出每組第一筆資料的 row（row_number == 0）
first_row_type = (
    remaining_with_rownum
    .filter(pl.col("row_number") == 0)
    .with_columns([
        pl.col("original_type").alias("first_type")
    ])
    .select(group_cols + ["first_type"])
)

# Step B: 找出那些 first_type != 0 的 group
first_type_nonzero = first_row_type.filter(pl.col("first_type") != 0)

# Step C: 再從原始資料中找這些 group 是否還有其他 type == 0 的資料
# （但不要算 row_number == 0，那是第一筆，已經驗證過不是 0）
potential_violations = (
    remaining_with_rownum
    .join(first_type_nonzero, on=group_cols, how="inner")
    .filter((pl.col("row_number") > 0) & (pl.col("original_type") == 0))
)

# 若違規筆數大於 0，才印出提醒
violation_count = potential_violations.shape[0]
if violation_count > 0:
    print(f"⚠️ 發現違反規則的資料筆數：{violation_count}")
    # 若你想看是哪幾組 group，也可以印出 unique groups：
    # print(potential_violations.select(group_cols).unique())
# =========================================================

# Step 3-2.5: 每組內，row_number > last_zero_row 的第一筆非 0
# 先合併 last_zero_row，然後篩選條件
post_last_zero = remaining_with_rownum.join(last_zero_index, on=group_cols, how="inner")
first_nonzero_after_last_zero = (
    post_last_zero
    .filter((pl.col("original_type") != 0) & (pl.col("row_number") > pl.col("last_zero_row")))
    .group_by(group_cols)
    .agg(pl.col("row_number").min().alias("first_nonzero_row"))
)

# 替代原本的 first_nonzero_index 與 join
range_limits = last_zero_index.join(first_nonzero_after_last_zero, on=group_cols, how="inner")


# 加入原資料
df_with_range = remaining_with_rownum.join(range_limits, on=group_cols, how="left")

# 抓出 row_number 在範圍內的資料
df_next_to_zero = df_with_range.filter(
    (pl.col("row_number") <= pl.col("first_nonzero_row")) &
    (pl.col("row_number") >= pl.col("last_zero_row"))
)

# Step 3-4: 確保 first_nonzero_row 剛好是最後一個非 0（才符合只接一筆非 0）
# 也就是在每組裡面，last_zero_row 後面不能有第二筆非 0
post_zero_nonzero_counts = (
    remaining_with_rownum
    .join(last_zero_index, on=group_cols, how="inner")
    .filter(
        (pl.col("row_number") > pl.col("last_zero_row")) &
        (pl.col("original_type") != 0)
    )
    .group_by(group_cols)
    .agg(pl.len().alias("nonzero_after_zero"))
)

# 篩選符合條件的 group（最後一筆 0 後面只有一筆非 0）
valid_r2_groups = post_zero_nonzero_counts.filter(
    pl.col("nonzero_after_zero") == 1
).select(group_cols)

# 取得符合條件的 r2_df
r2_df = df_next_to_zero.join(valid_r2_groups, on=group_cols, how="inner") \
                       .drop(["row_number", "last_zero_row", "first_nonzero_row", "original_type"])

# 額外統計：r2_df 中有幾組 group 是超過兩筆資料
r2_group_counts = r2_df.group_by(group_cols).count()
r2_more_than_2 = r2_group_counts.filter(pl.col("count") > 2).shape[0]


# Step 4: 計算剩餘資料（從 pl_df 中扣掉 r1_df + r2_df）
# 為了比較是否為重複資料，使用全部欄位進行 anti join
used_rows = pl.concat([r1_df, r2_df])
remaining_final = pl_df.join(used_rows, on=pl_df.columns, how="anti")

# 顯示結果
print(f"r1_df 筆數（整組都是 0）: {r1_df.shape[0]}")
print(f"r2_df 筆數（0 接 1 筆非 0）: {r2_df.shape[0]}")
print(f"未分類剩餘筆數（不屬於 r1/r2）: {remaining_final.shape[0]}")
print("=====")
print(f"原始輸入資料與此步驟分組資料筆數加總是否正確: { 1 if pl_df.shape[0]== r1_df.shape[0]+r2_df.shape[0]+remaining_final.shape[0] else 0}")
print(f"【例外情況】其中筆數超過兩筆的 group 數量: {r2_more_than_2}")

# 存成 CSV 檔案
r1_df.write_csv("data/clean_thread_r1.csv")
r2_df.write_csv("data/clean_thread_r2.csv")

print("已成功匯出 r1_df (clean_thread_r1.csv)與 r2_df (clean_thread_r2.csv) 為 CSV 檔案")


C:\Users\ChenChun\AppData\Local\Temp\ipykernel_8336\1339240898.py:127: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  r2_group_counts = r2_df.group_by(group_cols).count()


r1_df 筆數（整組都是 0）: 12108565
r2_df 筆數（0 接 1 筆非 0）: 10621366
未分類剩餘筆數（不屬於 r1/r2）: 8086551
=====
原始輸入資料與此步驟分組資料筆數加總是否正確: 1
【例外情況】其中筆數超過兩筆的 group 數量: 0
已成功匯出 r1_df (clean_thread_r1.csv)與 r2_df (clean_thread_r2.csv) 為 CSV 檔案


> 驗證 : 
> 1. df1 與 df2 的組內皆沒有，用 src_cre_datec 排序後 ，第一筆中 offr_type_id 不是 0 的資料

In [18]:
import polars as pl
r1_df =pl.read_csv("data/clean_thread_r1.csv")
r2_df =pl.read_csv("data/clean_thread_r2.csv")

In [19]:
group_keys = ["anon_slr_id", "anon_byr_id", "anon_item_id", "anon_thread_id"]

# Step 1：針對 df1 和 r2_df 分別處理
for name, df in [("r1_df", r1_df), ("r2_df", r2_df)]:
    # 為每個 group 加上 group 排序順序（用 src_cre_date 排）
    df_with_rank = (
        df.sort(group_keys + ["src_cre_date"])
        .with_columns([
            pl.arange(0, pl.len()).over(group_keys).alias("row_num")
        ])
    )

    # 取每組第一筆
    first_rows = df_with_rank.filter(pl.col("row_num") == 0)

    # 檢查第一筆中 offr_type_id != 0 的異常資料
    invalid_first = first_rows.filter(pl.col("offr_type_id") != 0)

    # 顯示結果
    print(f"\n{name} 中第一筆資料中 offr_type_id ≠ 0 的 group 數量：{invalid_first.height}")
    if invalid_first.height > 0:
        print(invalid_first.select(group_keys + ["src_cre_date", "offr_type_id"]).limit(5))



r1_df 中第一筆資料中 offr_type_id ≠ 0 的 group 數量：0

r2_df 中第一筆資料中 offr_type_id ≠ 0 的 group 數量：0


# Join 
- 此步驟要將已整理好的 clean_thread_r1.csv 及 clean_thread_r2.csv --(left join)--> clean_anon_bo_lists.csv
- 依據 anon_item_id 與 anon_slr_id 進行 inner join
- 結果輸入變數 merged_r1_df 及 merged_r2_df

In [20]:
# 載入主資料表（被 join 的左表）
bo_df = pl.read_csv(r"C:\Users\ChenChun\Downloads\bargaining_data\clean_anon_bo_lists.csv")

# 指定 join key
join_keys = ["anon_item_id", "anon_slr_id"]

# 執行 inner join
merged_r1_df = r1_df.join(bo_df, on=join_keys, how="inner")
merged_r2_df = r2_df.join(bo_df, on=join_keys, how="inner")

# 顯示合併後的資料筆數與型態
print("r1_df 筆數:", r1_df.height,"merged_r1_df 筆數:", merged_r1_df.height)
print("r2_df 筆數:", r2_df.height,"merged_r2_df 筆數:", merged_r2_df.height)


r1_df 筆數: 12108565 merged_r1_df 筆數: 12072061
r2_df 筆數: 10621366 merged_r2_df 筆數: 10607882


# join 後的資料清理
join 完成後，須根據以下條件清理資料:
  0. L1：排除 start_price_usd > 1000
  0. L2 : clean_list.csv 已事先完成
  1. T1 所有出價( offr_price )需低於刊登價格，也就是**（offr_price <= start_price_usd）**，若違反此規則
  2. ~~T2：賣家和買家均不可超過三次出價~~
  3. T3 若備標註為反向出價的資料( status_id == 7 )，必須有對應反價數據 (需要有下一筆資料)
  4. T4 被接受的報價( status_id == 0 or 6 )，不得有後續報價 
  5. T5 移除重複記錄

  最終輸出：只保留 不違反 T1 ~ T5 的**群組**，(若有一筆資料違反，則整組group key 一同刪除)

> 註: 之前清理 bo_lists 的條件:
> 1. 'anon_item_id', 'anon_slr_id' 皆不為空
> 2. ( 參考 L2 ) 若商品售出( item_price != N/A)，售價( item_price )不得大於商品列表的標價( start_price_usd )
> 3. 捨棄參考標價欄位(count …)

In [21]:
def clean_merged_offer_df(df: pl.DataFrame, label="") -> pl.DataFrame:
    group_cols = ["anon_item_id", "anon_slr_id", "anon_thread_id", "anon_byr_id"]
    
    # ========== Step L1：排除 start_price_usd > 1000 的群組 ==========
    df = df.with_columns([
        (pl.col("start_price_usd") > 1000).fill_null(False).alias("L1_flag")
    ])

    # 抓出有違規的群組
    l1_violation_groups = (
        df.filter(pl.col("L1_flag") == True)
        .select(group_cols)
        .unique()
    )

    if l1_violation_groups.height > 0:
        affected = df.join(l1_violation_groups, on=group_cols, how="semi").shape[0]
        print(f"🚫 {label} L1 限制：移除 start_price_usd > 1000 的群組，共刪除 {affected} 筆")
        df = df.join(l1_violation_groups, on=group_cols, how="anti")

    df = df.drop("L1_flag")  # 清理暫存欄位
    
    # ========== 資料排序 ==========
    df = df.sort(group_cols + ["src_cre_date"])
    
    # ========== Step T1：offr_price 不得高於 start_price_usd ==========
    df = df.with_columns([
        (pl.col("offr_price") > pl.col("start_price_usd")).fill_null(False).alias("T1_flag")
    ])

    # 統計 T1 違規 group
    group_stats = (
        df.group_by(group_cols)
        .agg([
            pl.col("T1_flag").any().alias("T1_flag"),
            pl.count().alias("group_size"),
            pl.sum("T1_flag").alias("T1_individual"),
        ])
    )

    t1_stats = group_stats.filter(pl.col("T1_flag") == True)
    t1_individual_removed = t1_stats.select(pl.sum("T1_individual")).item() if t1_stats.height > 0 else 0
    t1_group_total_removed = t1_stats.select(pl.sum("group_size")).item() if t1_stats.height > 0 else 0
    t1_group_removed = t1_group_total_removed - t1_individual_removed

    if t1_group_total_removed > 0:
        print(f"🚫 {label} T1 限制：刪除了 {t1_individual_removed} 筆資料，連帶刪除群組資料 {t1_group_removed} 筆，共刪除 {t1_group_total_removed} 筆")

    # 移除 T1 違規 group
    t1_bad_groups = t1_stats.select(group_cols)
    df = df.join(t1_bad_groups, on=group_cols, how="anti")
    
    # ========== Step T3：若某筆 status_id == 7，則該筆不能是 group 最後一筆 ==========
    # 為每筆資料加上 row_number 與 group_size
    df = df.sort(group_cols + ["src_cre_date"]).with_columns([
        pl.len().over(group_cols).alias("group_size"),
        pl.arange(0, pl.len()).over(group_cols).alias("row_number")
    ])

    # 找出 status_id == 7 且是 group 最後一筆的資料
    t3_violation_rows = df.filter(
        (pl.col("status_id") == 7) &
        (pl.col("row_number") == pl.col("group_size") - 1)
    )

    # 擷取這些違規資料所屬 group
    t3_violation_groups = t3_violation_rows.select(group_cols).unique()

    # 從原始 df 移除整個 group
    if t3_violation_groups.height > 0:
        affected = df.join(t3_violation_groups, on=group_cols, how="semi").shape[0]
        print(f"🚫 {label} T3 限制：移除最後一筆為 status_id==7 的群組，共刪除 {affected} 筆")
        df = df.join(t3_violation_groups, on=group_cols, how="anti")



    # ========== Step T4：若 group 中 status_id == 0/6 被接受，後面不得再出價 ==========
    df = df.sort(group_cols + ["src_cre_date"]).with_columns(
        pl.arange(0, pl.len()).over(group_cols).alias("row_number")
    )

    accepted = (
        df.filter(pl.col("status_id").is_in([0, 6]))
        .group_by(group_cols)
        .agg(pl.col("row_number").min().alias("accept_row"))
    )

    df_accept = df.join(accepted, on=group_cols, how="left")

    # 找出接受後仍出價的 group
    post_accepted_groups = (
        df_accept
        .filter(
            pl.col("row_number") > pl.col("accept_row")
        )
        .select(group_cols)
        .unique()
    )

    if post_accepted_groups.height > 0:
        affected = df_accept.join(post_accepted_groups, on=group_cols, how="semi").shape[0]
        print(f"🚫 {label} T4 限制：移除接受後仍出價的群組，共刪除 {affected} 筆")
        df_accept = df_accept.join(post_accepted_groups, on=group_cols, how="anti")

    df = df_accept.drop(["row_number", "accept_row"])

    # ========== Step T5：移除重複 ==========
    before = df.height
    df = df.unique()
    after = df.height
    if after < before:
        print(f"ℹ️ {label} T5：移除重複記錄 {before - after} 筆")

    return df


In [22]:
cleaned_r1_df = clean_merged_offer_df(merged_r1_df, label="R1")
cleaned_r2_df = clean_merged_offer_df(merged_r2_df, label="R2")


🚫 R1 L1 限制：移除 start_price_usd > 1000 的群組，共刪除 689691 筆


C:\Users\ChenChun\AppData\Local\Temp\ipykernel_8336\2734003073.py:36: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias("group_size"),


🚫 R1 T1 限制：刪除了 38160 筆資料，連帶刪除群組資料 3495 筆，共刪除 41655 筆
🚫 R1 T3 限制：移除最後一筆為 status_id==7 的群組，共刪除 346 筆
🚫 R1 T4 限制：移除接受後仍出價的群組，共刪除 346325 筆
🚫 R2 L1 限制：移除 start_price_usd > 1000 的群組，共刪除 730904 筆
🚫 R2 T1 限制：刪除了 251340 筆資料，連帶刪除群組資料 197564 筆，共刪除 448904 筆
🚫 R2 T3 限制：移除最後一筆為 status_id==7 的群組，共刪除 164 筆
🚫 R2 T4 限制：移除接受後仍出價的群組，共刪除 4636 筆


> 檢查

In [23]:
# 檢查 R1 中 group_size 不等於 1 的資料
r1_invalid = cleaned_r1_df.filter(pl.col("group_size") != 1)
print(f"列出 R1 中 group_size 不等於 1 的資料: {r1_invalid.height}")
# 列出 R1 中 group_size 不等於 1 的資料
# print(r1_invalid.select(["anon_item_id", "anon_slr_id", "anon_thread_id", "anon_byr_id", "group_size"]))

# 檢查 R2 中 group_size 不等於 2 的資料  
r2_invalid = cleaned_r2_df.filter(pl.col("group_size") != 2)
print(f"R2 中 group_size 不等於 2 的資料  : {r2_invalid.height}")

列出 R1 中 group_size 不等於 1 的資料: 822368
R2 中 group_size 不等於 2 的資料  : 0


刪去多於欄位並儲存

In [24]:
print(len(cleaned_r1_df))
print(len(cleaned_r2_df))
cleaned_r1_df = cleaned_r1_df.drop(['T1_flag','group_size'])
cleaned_r2_df = cleaned_r2_df.drop(['T1_flag','group_size'])
# 可選擇匯出結果
cleaned_r1_df.write_csv("data/cleaned_r1_joined.csv")
cleaned_r2_df.write_csv("data/cleaned_r2_joined.csv")

10994044
9423274


# Variable 處理
> 0415 改成: SRP_0是原始商品標價

1. round1 (cleaned_r1_joined.csv): 每組都僅一列資料
    需要新增的欄位:
    - BRP_0: 買家初報價
    - SRT_0: 賣家回覆時間(單位:秒)
    - SRP_0: 賣家價格
    - SA_0: 賣家是否同意(0/1)

2. round2 (cleaned_r1_joined.csv):  每組有兩列資料
    - type 1. 賣家拒絕，並反報價
        需要新增的欄位:
        - 第一筆：BRP_0, SRT_0
        - 第二筆 :  SRP_1,BRT_1,BA_1

    - type 2. 賣家僅拒絕，沒有反報價，買家二次報價
        需要新增的欄位:
        - 第一筆 :  BRP_0, SRT_0
        - 第二筆 :  BRT_1, BRP_1, SRT_2  ,SA_1 


## 1. round1 (cleaned_r1_joined.csv): 每組都僅一列資料
- 新建立一個欄位 `BRP_0` :
    - BRP_0 = `start_price_usd` 減去 `offr_price`

- 新欄位 `SRT_0`(單位:秒) : 
    - status_id == 0 (報價過期)，SRT_0 為 48 小時
    - ~~status_id == 7 (反報價)，則 SRT_0 為下一筆資料的`src_cre_date` 減去 此筆資料的`src_cre_date`~~
    - status_id 為  1,2,8 (接受/拒絕/其他買家被接受) ，SRT_0 為此筆資料 (`response_time - src_cre_date`)
    - ~~status_id 為  6,9 ，SRT_0 為 0~~

- 新欄位`SA_0`(Seller Acceptance): 
    - status_id == 0,2,6,7,8 ，此欄位為 0 
    - status_id == 1,9 ，此欄位為 1 
  
- 新欄位`SRP_0`:
    - 商品初始價格 `start_price_usd`

In [26]:
import polars as pl

# 載入 Round1 資料
df = pl.read_csv("data/cleaned_r1_joined.csv")

In [28]:


df = df.with_columns([
    pl.col("src_cre_date").str.strptime(pl.Datetime, strict=False),
    pl.col("response_time").str.strptime(pl.Datetime, strict=False),
])

# 加入 BRP_0 欄位
df = df.with_columns([
    (pl.col("start_price_usd") - pl.col("offr_price")).alias("BRP_0")
])

# 加入 SRT_0：根據 status_id 決定秒數
df = df.with_columns([
    pl.when(pl.col("status_id") == 0)
      .then(48 * 3600)
      .when(pl.col("status_id").is_in([1, 2, 8]))
      .then(
          ((pl.col("response_time") - pl.col("src_cre_date"))
           .dt.total_nanoseconds() / 1_000_000_000).cast(pl.Int32)
      )
      .otherwise(None)  # 更合理地回傳 Null，而不是 0
      .alias("SRT_0")
])


# 加入 SA_0
df = df.with_columns([
    pl.when(pl.col("status_id").is_in([1, 9]))
      .then(1)
      .otherwise(0)
      .alias("SA_0")
])

# 加入 SRP_0
df = df.with_columns([
    pl.col("start_price_usd").alias("SRP_0")
])

# 匯出處理結果
df.write_csv("data/round1_done.csv")
print("round1_done.csv 已完成並匯出")


round1_done.csv 已完成並匯出


> 檢查
針對以下欄位做檢查：

| 欄位    | 合理性檢查說明 |
|---------|----------------|
| `BRP_0` | 不應為極端大值（如 > 1000）、不應大量為 null |
| `SRP_0` | 應該與 "start_price_usd" 相同
| `SA_0`  | 應該同時有 0 與 1，若只有一種值 → 不合理 |
| `SRT_0` | 通常應該 >= 60 秒，若出現大量 < 1 秒或負值 → 不合理 |

In [29]:
df = pl.read_csv("data/round1_done.csv")

# 1. 檢查 BRP_0 是否有超過 1000、是否大量為 null
brp_stats = df.select([
    pl.col("BRP_0").null_count().alias("null_BRP"),
    pl.col("BRP_0").mean().alias("mean_BRP"),
    pl.col("BRP_0").std().alias("std_BRP"),
])

# 2. 檢查 SRP_0 是否等於 start_price_usd
identical_check = df.with_columns([
    (pl.col("SRP_0") == pl.col("start_price_usd")).alias("is_identical")
])
# 統計不同的情況
identity_stats = identical_check.select([
    pl.sum("is_identical").alias("相同值數量"),
    (pl.count() - pl.sum("is_identical")).alias("不相同值數量"),
    (pl.sum("is_identical") / pl.count() * 100).alias("相同值百分比")
])

# 3. 檢查 SA_0 是否只有 0 或 1
sa_stats = df.select([
    (pl.col("SA_0") == 0).sum().alias("SA_0_zeros"),
    (pl.col("SA_0") == 1).sum().alias("SA_0_ones"),
])

# 4. 檢查 SRT_0 是否有過短（<1秒）或為負值
srt_stats = df.select([
    (pl.col("SRT_0") < 1).sum().alias("SRT_0_lt_1s"),
    (pl.col("SRT_0") == 0).sum().alias("SRT_0_is0"),
    (pl.col("SRT_0") < 0).sum().alias("SRT_0_negative"),
    pl.col("SRT_0").max().alias("max_SRT"),
    pl.col("SRT_0").mean().alias("mean_SRT")
])

print("========= BRP_0 統計:")
stats = df.select([
    pl.col("BRP_0").max().alias("max_BRP"),
    pl.col("BRP_0").mean().alias("mean_BRP")
]).to_dicts()[0]

print(f"最大 BRP: {stats['max_BRP']:.2f}")
print(f"平均 BRP: {stats['mean_BRP']:.2f}")
print(brp_stats)

print("\n========= SRP_0 確認:")
print(identity_stats)

print("\n========= SA_0 分布:")
print(sa_stats)

print("\n========= SRT_0 分布:")
print(srt_stats)


========= BRP_0 統計:
最大 BRP: 999.01
平均 BRP: 50.49
shape: (1, 3)
┌──────────┬───────────┬───────────┐
│ null_BRP ┆ mean_BRP  ┆ std_BRP   │
│ ---      ┆ ---       ┆ ---       │
│ u32      ┆ f64       ┆ f64       │
╞══════════╪═══════════╪═══════════╡
│ 0        ┆ 50.494412 ┆ 93.525258 │
└──────────┴───────────┴───────────┘

========= SRP_0 確認:
shape: (1, 3)
┌────────────┬──────────────┬──────────────┐
│ 相同值數量 ┆ 不相同值數量 ┆ 相同值百分比 │
│ ---        ┆ ---          ┆ ---          │
│ u32        ┆ u32          ┆ f64          │
╞════════════╪══════════════╪══════════════╡
│ 10994044   ┆ 0            ┆ 100.0        │
└────────────┴──────────────┴──────────────┘

========= SA_0 分布:
shape: (1, 2)
┌────────────┬───────────┐
│ SA_0_zeros ┆ SA_0_ones │
│ ---        ┆ ---       │
│ u32        ┆ u32       │
╞════════════╪═══════════╡
│ 4749659    ┆ 6244385   │
└────────────┴───────────┘

========= SRT_0 分布:
shape: (1, 5)
┌─────────────┬───────────┬────────────────┬─────────┬──────────────┐
│ SRT_0_lt_1s ┆ S

C:\Users\ChenChun\AppData\Local\Temp\ipykernel_8336\3465950016.py:17: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  (pl.count() - pl.sum("is_identical")).alias("不相同值數量"),
C:\Users\ChenChun\AppData\Local\Temp\ipykernel_8336\3465950016.py:18: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  (pl.sum("is_identical") / pl.count() * 100).alias("相同值百分比")


> 檢查SRT_0 > 48小時的紀錄

In [30]:
# 檢查 SRT_0 > 172800 (48小時) 的資料
abnormal_srt = df.filter(pl.col("SRT_0") > 172800)

# 顯示異常資料的筆數
print(f"SRT_0 超過 48 小時 (172800 秒) 的記錄數: {abnormal_srt.height}")

# 使用正確的ISO格式進行轉換
df = df.with_columns([
    pl.col("src_cre_date").str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%S.%f").alias("src_cre_date"),
    pl.col("response_time").filter(pl.col("response_time").is_not_null())
      .str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%S.%f").alias("response_time")
])

# 轉換後，顯示異常記錄的詳細信息
abnormal_srt = df.filter(pl.col("SRT_0") > 172800)
if abnormal_srt.height > 0:
    print("\n異常資料樣本:")
    # 選擇可用的欄位
    available_cols = [col for col in ["SRT_0", "anon_thread_id", "anon_slr_id", "anon_byr_id", "anon_item_id"] 
                     if col in abnormal_srt.columns]
    
    print(abnormal_srt.select(available_cols).sort("SRT_0", descending=True).head(10))

C:\Users\ChenChun\AppData\Local\Temp\ipykernel_8336\1696487933.py:9: ChronoFormatWarning: Detected the pattern `.%f` in the chrono format string. This pattern should not be used to parse values after a decimal point. Use `%.f` instead. See the full specification: https://docs.rs/chrono/latest/chrono/format/strftime
  pl.col("src_cre_date").str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%S.%f").alias("src_cre_date"),
C:\Users\ChenChun\AppData\Local\Temp\ipykernel_8336\1696487933.py:10: ChronoFormatWarning: Detected the pattern `.%f` in the chrono format string. This pattern should not be used to parse values after a decimal point. Use `%.f` instead. See the full specification: https://docs.rs/chrono/latest/chrono/format/strftime
  pl.col("response_time").filter(pl.col("response_time").is_not_null())


SRT_0 超過 48 小時 (172800 秒) 的記錄數: 1182

異常資料樣本:
shape: (10, 5)
┌────────┬────────────────┬─────────────┬─────────────┬──────────────┐
│ SRT_0  ┆ anon_thread_id ┆ anon_slr_id ┆ anon_byr_id ┆ anon_item_id │
│ ---    ┆ ---            ┆ ---         ┆ ---         ┆ ---          │
│ i64    ┆ i64            ┆ i64         ┆ i64         ┆ i64          │
╞════════╪════════════════╪═════════════╪═════════════╪══════════════╡
│ 336248 ┆ 7387830        ┆ 2226650     ┆ 602076      ┆ 68328749     │
│ 332313 ┆ 10951974       ┆ 5435235     ┆ 5711358     ┆ 8671325      │
│ 329145 ┆ 4672182        ┆ 625801      ┆ 9761812     ┆ 94027845     │
│ 329125 ┆ 1033529        ┆ 1311544     ┆ 9742284     ┆ 86879973     │
│ 322731 ┆ 11831385       ┆ 1206777     ┆ 2912888     ┆ 56970751     │
│ 322485 ┆ 12098952       ┆ 2359264     ┆ 6186306     ┆ 55750189     │
│ 317500 ┆ 2567275        ┆ 5070189     ┆ 6641468     ┆ 50015835     │
│ 310125 ┆ 3265650        ┆ 4106421     ┆ 228889      ┆ 35203219     │
│ 309058 ┆ 11444

## 2. round2 (cleaned_r1_joined.csv):  每組有兩列資料

1. type 1. 賣家拒絕，並反報價 ( 群組第二筆資料的 offr_type_id == 2 )
    需要新增的欄位:
    - 第一筆：BRP_0, SRT_0, SRP_0
      - `BRP` :`start_price_usd` 減去 `offr_price`
      - `SRT`(單位:秒) : 
          - status_id == 0 (報價過期)，SRT_0 為 48 小時
          - status_id == 7 (反報價)，則 SRT_0 為下一筆資料的`src_cre_date` 減去 此筆資料的`src_cre_date`
          - status_id 為  1,2,8 (接受/拒絕/其他買家被接受) ，SRT_0 為此筆資料 (`response_time - src_cre_date`)
          - ~~status_id 為  6,9 ，SRT_0 為 0~~
      - `SRP_0`:`start_price_usd`
    - 第二筆 :  SRP_1,BRT_1,BA_1
      - `SRP`: `start_price_usd` 減去 `offr_price`
      - `BRT`(單位:秒) : 
          - status_id == 0 (報價過期)，SRT_0 為 48 小時
          - ~~status_id == 7 (反報價)，則 SRT_0 為下一筆資料的`src_cre_date` 減去 此筆資料的`src_cre_date`~~
          - status_id 為  1,2,8 (接受/拒絕/其他買家被接受) ，SRT_0 為此筆資料 (`response_time - src_cre_date`)
          - ~~status_id 為  6,9 ，SRT_0 為 0~~
      - `BA` :
        - status_id == 0,2,6,7,8 (拒絕)，此欄位為 0 
        - status_id == 1,9 (接受)，此欄位為 1 


    

2. type 2. 賣家僅拒絕，沒有反報價，買家二次報價 ( 群組第二筆資料的 offr_type_id == 1 )
    需要新增的欄位:
    - 第一筆 :  BRP_0, SRT_0, SRP_0
      - `BRP` :`start_price_usd` 減去 `offr_price`
      - `SRT`(單位:秒) : 
          - status_id == 0 (報價過期)，SRT_0 為 48 小時
          - status_id == 7 (反報價)，則 SRT_0 為下一筆資料的`src_cre_date` 減去 此筆資料的`src_cre_date`
          - status_id 為  1,2,8 (接受/拒絕/其他買家被接受) ，SRT_0 為此筆資料 (`response_time - src_cre_date`)
          - ~~status_id 為  6,9 ，SRT_0 為 0~~
      - `SRP_0`:`start_price_usd`
    - 第二筆 :  BRT_1, BRP_1, SRT_2  ,SA_1 
      - `BRT`(單位:秒) :此筆資料的`src_cre_date` 減去 上一筆資料的`response_time`
      - `BRP` :`start_price_usd` 減去 `offr_price`
      - `SRT`(單位:秒) : 
          - status_id == 0 (報價過期)，SRT_0 為 48 小時
          - ~~status_id == 7 (反報價)，則 SRT_0 為下一筆資料的`src_cre_date` 減去 此筆資料的`src_cre_date`~~
          - status_id 為  1,2,8 (接受/拒絕/其他買家被接受) ，SRT_0 為此筆資料 (`response_time - src_cre_date`)
          - ~~status_id 為  6,9 ，SRT_0 為 0~~
      - `SA` :
        - status_id == 0,2,6,7,8 (拒絕)，此欄位為 0 
        - status_id == 1,9 (接受)，此欄位為 1 


In [31]:
import polars as pl

# 載入 Round1 資料
df = pl.read_csv("data/cleaned_r2_joined.csv")

df.select([
    pl.col("start_price_usd").null_count().alias("null_start_price_usd"),
    pl.col("offr_price").null_count().alias("null_offr_price")
])


df = df.with_columns([
    pl.col("src_cre_date").str.strptime(pl.Datetime, strict=False),
    pl.col("response_time").str.strptime(pl.Datetime, strict=False),
])

# 定義分組欄位並排序（組內依時間）
group_cols = ["anon_item_id", "anon_slr_id", "anon_thread_id", "anon_byr_id"]
df = df.sort(group_cols + ["src_cre_date"])

# 加上 group_index（組內排序）
df = df.with_columns([
    pl.arange(0, pl.len()).over(group_cols).alias("group_index")
])

# 將第二筆的 offr_type_id 抓出來當群組分類依據
group_type_df = (
    df.filter(pl.col("group_index") == 1)
    .select(group_cols + [pl.col("offr_type_id").alias("second_offr_type_id")])
)
df = df.join(group_type_df, on=group_cols, how="left")

# 分出 type1 和 type2
type1_df = df.filter(pl.col("second_offr_type_id") == 2)
type2_df = df.filter(pl.col("second_offr_type_id") == 1)

# 印出 type1_df 與 type2_df的資料筆數
print(f"Type 1 資料筆數: {type1_df.shape[0]}")
print(f"Type 2 資料筆數: {type2_df.shape[0]}")

Type 1 資料筆數: 9423274
Type 2 資料筆數: 0


In [32]:
type1_df = type1_df.with_columns([
    pl.col("src_cre_date").shift(-1).over(group_cols).alias("next_src_cre_date")
])


# 定義 SRT 計算函式
def compute_srt(status_col, response_col, src_col, next_src_col):
    return (
        pl.when(status_col == 0)
        .then(48 * 3600)
        .when(status_col == 7)
        .then(((next_src_col - src_col).dt.total_nanoseconds() / 1_000_000_000).cast(pl.Int32))
        .when(status_col.is_in([1, 2, 8]))
        .then(((response_col - src_col).dt.total_nanoseconds() / 1_000_000_000).cast(pl.Int32))
        .otherwise(None)
    )


# ➤ Type 1 處理
type1 = type1_df.with_columns([
    # 第一筆
    pl.when(pl.col("group_index") == 0)
      .then(pl.col("start_price_usd") - pl.col("offr_price"))
      .otherwise(None)
      .alias("BRP_0"),

    pl.when(pl.col("group_index") == 0)
    .then(compute_srt(
        pl.col("status_id"),
        pl.col("response_time"),
        pl.col("src_cre_date"),
        pl.col("next_src_cre_date")
    ))
    .otherwise(None)
    .alias("SRT_0"),
    
    pl.when(pl.col("group_index") == 0)
    .then(pl.col("start_price_usd"))
    .otherwise(None)  # 明確處理其他情況
    .alias("SRP_0"),

    # 第二筆
    pl.when(pl.col("group_index") == 1)
      .then(pl.col("start_price_usd") - pl.col("offr_price"))
      .otherwise(None)
      .alias("SRP_1"),

    pl.when(pl.col("group_index") == 1)
    .then(compute_srt(
        pl.col("status_id"),
        pl.col("response_time"),
        pl.col("src_cre_date"),
        pl.col("next_src_cre_date")
    ))
    .otherwise(None)
    .alias("BRT_1"),


    pl.when(pl.col("group_index") == 1)
      .then(pl.when(pl.col("status_id").is_in([1, 9])).then(1).otherwise(0))
      .otherwise(None)
      .alias("BA_1"),
])


# # ➤ Type 2 處理（需要加入前一筆 response_time）
# # 先新增前一筆 response_time
# type2_df = type2_df.with_columns([
#     pl.col("response_time")
#       .shift(1)
#       .over(group_cols)
#       .alias("prev_response_time")
# ])

# type2 = type2_df.with_columns([
#     # 第一筆
#     pl.when(pl.col("group_index") == 0)
#       .then(pl.col("start_price_usd") - pl.col("offr_price"))
#       .otherwise(None)
#       .alias("BRP_0"),

#     pl.when(pl.col("group_index") == 0)
#       .then(compute_srt(pl.col("status_id"), pl.col("response_time"), pl.col("src_cre_date")))
#       .otherwise(None)
#       .alias("SRT_0"),

#     # 第二筆
#     pl.when(pl.col("group_index") == 1)
#       .then(((pl.col("src_cre_date") - pl.col("prev_response_time")).dt.total_nanoseconds() / 1_000_000_000).cast(pl.Int32))
#       .otherwise(None)
#       .alias("BRT_1"),

#     pl.when(pl.col("group_index") == 1)
#       .then(pl.col("start_price_usd") - pl.col("offr_price"))
#       .otherwise(None)
#       .alias("BRP_1"),

#     pl.when(pl.col("group_index") == 1)
#       .then(compute_srt(pl.col("status_id"), pl.col("response_time"), pl.col("src_cre_date")))
#       .otherwise(None)
#       .alias("SRT_2"),

#     pl.when(pl.col("group_index") == 1)
#       .then(pl.when(pl.col("status_id").is_in([1, 9])).then(1).otherwise(0))
#       .otherwise(None)
#       .alias("SA_1"),
# ])


# 合併輸出結果（可視需要寫成 csv）
# 匯出 Type1 資料
type1.write_csv("data/round2_type1.csv")
print("round2_type1.csv 已輸出")

# # 匯出 Type2 資料
# type2.write_csv("round2_type2.csv")
# print("round2_type2.csv 已輸出")




round2_type1.csv 已輸出


> 檢查個欄位合理性

In [33]:
import polars as pl

# 讀取資料
type1_df = pl.read_csv("data/round2_type1.csv")

print("總資料筆數: ",len(type1_df))
print("每組有兩筆資料，一半欄位應該為 Null，4711637*2=",4711637*2)
# 檢查欄位是否缺值
def check_columns(df, label, expected_cols):
    print(f"\n📊 合理性檢查 - {label}")
    for col in expected_cols:
        if col in df.columns:
            null_count = df.filter(pl.col(col).is_null()).shape[0]
            print(f"🔍 欄位 {col} - Null 數量: {null_count}")
        else:
            print(f"⚠️ 欄位 {col} 缺失！")

# 檢查是否秒數過短（小於1秒）
def check_duration_issues(df, label, duration_cols):
    for col in duration_cols:
        if col in df.columns:
            too_fast = df.filter(pl.col(col) < 1).shape[0]
            print(f"⚠️ 欄位 {col} 有 {too_fast} 筆小於 1 秒的資料")

# 檢查 0/1 分布
def check_binary_distribution(df, label, binary_cols):
    for col in binary_cols:
        if col in df.columns:
            print(f"\n✅ 欄位 {col} 數值分布:")
            print(df.group_by(col).count().sort(col))

# 預期欄位定義
type1_cols = ["BRP_0", "SRT_0", "SRP_1", "BRT_1", "BA_1"]
# type2_cols = ["BRP_0", "SRT_0", "BRT_1", "BRP_1", "SRT_2", "SA_1"]

# 執行檢查
check_columns(type1_df, "Type 1", type1_cols)
# check_columns(type2_df, "Type 2", type2_cols)

check_duration_issues(type1_df, "Type 1", ["SRT_0", "BRT_1"])
# check_duration_issues(type2_df, "Type 2", ["SRT_0", "BRT_1", "SRT_2"])

check_binary_distribution(type1_df, "Type 1", ["BA_1"])
# check_binary_distribution(type2_df, "Type 2", ["SA_1"])


總資料筆數:  9423274
每組有兩筆資料，一半欄位應該為 Null，4711637*2= 9423274

📊 合理性檢查 - Type 1
🔍 欄位 BRP_0 - Null 數量: 4711637
🔍 欄位 SRT_0 - Null 數量: 4711637
🔍 欄位 SRP_1 - Null 數量: 4711637
🔍 欄位 BRT_1 - Null 數量: 4711637
🔍 欄位 BA_1 - Null 數量: 4711637
⚠️ 欄位 SRT_0 有 0 筆小於 1 秒的資料
⚠️ 欄位 BRT_1 有 0 筆小於 1 秒的資料

✅ 欄位 BA_1 數值分布:
shape: (3, 2)
┌──────┬─────────┐
│ BA_1 ┆ count   │
│ ---  ┆ ---     │
│ i64  ┆ u32     │
╞══════╪═════════╡
│ null ┆ 4711637 │
│ 0    ┆ 3649768 │
│ 1    ┆ 1061869 │
└──────┴─────────┘


C:\Users\ChenChun\AppData\Local\Temp\ipykernel_8336\1278563784.py:30: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  print(df.group_by(col).count().sort(col))
